In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks/NeuralNetwork")

In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import sys

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

import numpy as np
import random
import copy
import torch.optim as optim
import pickle
import os
import pandas as pd
import time
import matplotlib.pyplot as plt


In [2]:
sys.path.append("/Volumes/GoogleDrive/My Drive/Colab Notebooks/NeuralNetwork")

In [3]:
from library_model import layers as lay
from library_model import model_building as mb
from library_model import model_training as mt
from data import data_loading as dt
from data import text as txt

In [ ]:
%%bash
pip install torchdata

In [ ]:
%%bash
pip install torchtext

In [ ]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')

In [4]:
states = [copy.deepcopy(lay.Network_state()) for _ in range(5)]
for k, state in enumerate(states):
    p= state.parameters
    tr= state.training

    p.d_model=200
    p.d_hid=200
    p.nheads=2
    p.d_key = p.d_model //p.nheads
    p.nlayers=2
    p.network = "encoder"


    tr.batch_size=20
    tr.data_fraction = 0.8
    tr.lr = 1.
    tr.seq_length=35
    tr.optimizer = "sgd"
    tr.schedule = mb.learning_rate_step(5., 20/19, 1)

In [5]:
lay.param_count(states[1])

480000

In [ ]:
d = dt.Data()
d.tokenizer, d.train_dataloader, d.test_dataloader = dt.process_data(train_iter, txt.library_text_coders, state, network = "encoder") #switch to transformer if interested in encoder/decoder model
p.ntokens = p.ntokens_out = len(d.tokenizer.vocab)

In [ ]:
parts = mb.get_transformer_parts(state)
model = lay.EncoderModel(parts.encoder, parts.linear).to(state.device)

In [ ]:
opt, scheduler = mb.get_optimizer(state, model)
criterion = nn.CrossEntropyLoss()
train = mt.Model_training(model, opt, criterion, scheduler)